<a href="https://colab.research.google.com/github/rajindermath/Crop-Classification/blob/master/02-03-2021-Laboro_Tomato_Instance_seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#%cd drive/MyDrive/28-02-2021-hazal
#%cd /content/drive/MyDrive/Caterpillers
%cd /content/drive/MyDrive/laborotemp


/content/drive/MyDrive/laborotemp


In [ ]:
!pip install -U torch torchvision
!pip install git+https://github.com/facebookresearch/fvcore.git
import torch, torchvision
torch.__version__

In [ ]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
#%cd /content/drive/MyDrive/28-02-2021-hazal
#%cd /content/drive/MyDrive/Caterpillers
%cd /content/drive/MyDrive/laborotemp



/content/drive/MyDrive/laborotemp


In [ ]:
from detectron2.data.datasets import register_coco_instances
#register_coco_instances("fruits_nuts", {}, "./data/trainval.json", "./data/images")
#register_coco_instances("fruits_nuts", {}, "./train.json", "./train")
register_coco_instances("laboro_train", {}, "./laboro_big/train.json", "./laboro_big/train")
register_coco_instances("laboro_test", {}, "./laboro_big/test.json", "./laboro_big/test")


In [ ]:
#fruits_nuts_metadata = MetadataCatalog.get("fruits_nuts")
#dataset_dicts = DatasetCatalog.get("fruits_nuts")

fruits_nuts_metadata = MetadataCatalog.get("laboro_train")
dataset_dicts = DatasetCatalog.get("laboro_train")

[03/02 09:02:39 d2.data.datasets.coco]: Loaded 353 images in COCO format from ./laboro_big/train.json


In [ ]:
# EXTRA

#Implementing my own Trainer Module here to use the COCO validation evaluation during training
# TODO: add data custom augmentation 
from detectron2.engine import DefaultTrainer
class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
import random

for d in random.sample(dataset_dicts, 10):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=fruits_nuts_metadata, scale=0.2)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
# To find out inconsistent CUDA versions, if there is not "failed" word in this output then things are fine.
!python -m detectron2.utils.collect_env

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg = get_cfg()
cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("laboro_train",)
#cfg.DATASETS.TEST = ("laboro_test",)   # no metrics implemented for this dataset
cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 3000    # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # 3 classes (data, fig, hazelnut)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
#trainer = DefaultTrainer(cfg)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()




In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
#from detectron2.data import build_detection_test_loader
#evaluator = COCOEvaluator("laboro_test", cfg, False, output_dir="./output/")
#val_loader = build_detection_test_loader(cfg, "laboro_test")
#print(inference_on_dataset(trainer.model, val_loader, evaluator))


#####
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, LVISEvaluator
from detectron2.data import build_detection_test_loader

evaluator = COCOEvaluator("laboro_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "laboro_test")
inference_on_dataset(trainer.model, val_loader, evaluator)
# another equivalent way is to use trainer.test



In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("laboro_test", )
predictor = DefaultPredictor(cfg)

In [ ]:
val_dataset_dicts = DatasetCatalog.get("laboro_test")
val_metadata_dicts = MetadataCatalog.get("laboro_test")

[03/02 10:50:28 d2.data.datasets.coco]: Loaded 89 images in COCO format from ./laboro_big/test.json


In [ ]:
from detectron2.utils.visualizer import ColorMode

for d in random.sample(dataset_dicts, 30):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=val_metadata_dicts, 
                   scale=0.2, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
fruits_nuts_metadata

In [ ]:
import time
times = []
for i in range(20):
    start_time = time.time()
    outputs = predictor(im)
    delta = time.time() - start_time
    times.append(delta)
mean_delta = np.array(times).mean()
fps = 1 / mean_delta
print("Average(sec):{:.2f},fps:{:.2f}".format(mean_delta, fps))

Average(sec):0.19,fps:5.30


In [ ]:
import io
import imageio
fig, ax = plt.subplots(2, 2, figsize =(20,15))
#plt.axis('off')
indices=[ax[0][0],ax[1][0],ax[0][1],ax[1][1] ]
i=-1
for d in random.sample(val_dataset_dicts, 4):
    i=i+1    
    im = imageio.imread(d["file_name"])
    outputs = predictor(im)
    #plt.axis('off')
    v = Visualizer(im[:, :, ::-1],
                   metadata=val_metadata_dicts, 
                   scale=0.2, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    indices[i].grid(False)
    indices[i].axis(False)
    indices[i].imshow(out.get_image()[:, :, ::-1])
    #plt.axis('off')